In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.data.experimental import AUTOTUNE

In [24]:
raw_data = pd.read_csv('train.csv')
raw_data[:5]
# pclass = Ticket class
# sibsp = # of siblings / spouses aboard the Titanic
# parch = # of parents / children aboard the Titanic
# ticket = Ticket number
# fare = Passenger fare
# cabin = Cabin number
# embarked = Port of Embarkation C = Cherbourg, Q = Queenstown, S = Southampton

# Features:
# Categorical input : (we will apply one hot encoding on them) 
# Pclass - Name (only last name), Sex, Age, Cabin (see what to extract from it), Embarked, Ticket
# Age might be better with binning
# Ticket: extract relevant info

# Numerical input:
# SibSp, Parch, Fare (needs normalisation)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [25]:
print('Survived = ' + str(raw_data[raw_data.Survived == 1].PassengerId.count()))
print('Died = ' + str(raw_data[raw_data.Survived == 0].PassengerId.count()))
# TODO: Is the skew in Died / Survived big? 

Survived = 342
Died = 549


In [26]:
raw_data['Fare'] = raw_data['Fare'] / 512 # the only column to normalize

# Create one-hot encodings
raw_data = pd.concat([raw_data, pd.get_dummies(raw_data.Pclass, prefix='Pclass_')], axis=1)
raw_data = pd.concat([raw_data, pd.get_dummies(raw_data.Embarked, prefix='Embarked_')], axis=1)
raw_data = pd.concat([raw_data, pd.get_dummies(raw_data.Sex, prefix='Sex_')], axis=1)

In [27]:
feature_columns = ['SibSp', 'Parch', 'Fare']

all_columns = raw_data.columns.values.tolist()
feature_columns += list(filter(lambda x : x.startswith('Pclass_'), all_columns))
feature_columns += list(filter(lambda x : x.startswith('Embarked_'), all_columns))
feature_columns += list(filter(lambda x : x.startswith('Sex_'), all_columns))

feature_columns

['SibSp',
 'Parch',
 'Fare',
 'Pclass__1',
 'Pclass__2',
 'Pclass__3',
 'Embarked__C',
 'Embarked__Q',
 'Embarked__S',
 'Sex__female',
 'Sex__male']

In [28]:
raw_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Pclass__1,Pclass__2,Pclass__3,Embarked__C,Embarked__Q,Embarked__S,Sex__female,Sex__male
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,0.062899,0.242424,0.206510,0.551066,0.188552,0.086420,0.722783,0.352413,0.647587
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,0.097057,0.428790,0.405028,0.497665,0.391372,0.281141,0.447876,0.477990,0.477990
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,0.015450,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,0.028231,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,0.060547,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,1.000643,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [31]:
raw_data.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Pclass__1,Pclass__2,Pclass__3,Embarked__C,Embarked__Q,Embarked__S,Sex__female,Sex__male
PassengerId,1.000000,-0.005007,-0.035144,0.036847,-0.057527,-0.001652,0.012658,0.034303,-0.000086,-0.029486,-0.001205,-0.033606,0.022148,-0.042939,0.042939
Survived,-0.005007,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307,0.285904,0.093349,-0.322308,0.168240,0.003650,-0.155660,0.543351,-0.543351
Pclass,-0.035144,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500,-0.885924,-0.188432,0.916673,-0.243292,0.221009,0.081720,-0.131900,0.131900
Age,0.036847,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067,0.348941,0.006954,-0.312271,0.036261,-0.022405,-0.032523,-0.093254,0.093254
SibSp,-0.057527,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651,-0.054582,-0.055932,0.092548,-0.059528,-0.026354,0.070941,0.114631,-0.114631
Parch,-0.001652,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225,-0.017633,-0.000734,0.015790,-0.011069,-0.081228,0.063036,0.245489,-0.245489
Fare,0.012658,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000,0.591711,-0.118557,-0.413333,0.269335,-0.117216,-0.166603,0.182333,-0.182333
Pclass__1,0.034303,0.285904,-0.885924,0.348941,-0.054582,-0.017633,0.591711,1.000000,-0.288585,-0.626738,0.296423,-0.155342,-0.170379,0.098013,-0.098013
Pclass__2,-0.000086,0.093349,-0.188432,0.006954,-0.055932,-0.000734,-0.118557,-0.288585,1.000000,-0.565210,-0.125416,-0.127301,0.192061,0.064746,-0.064746
Pclass__3,-0.029486,-0.322308,0.916673,-0.312271,0.092548,0.015790,-0.413333,-0.626738,-0.565210,1.000000,-0.153329,0.237449,-0.009511,-0.137143,0.137143


In [32]:
data = raw_data[feature_columns]

training_examples = data[:-150]
training_targets = raw_data['Survived'][:-150]

testing_examples = data[-150:]
testing_targets = raw_data['Survived'][-150:]

In [36]:
model = keras.Sequential([
    keras.layers.Dense(3, input_shape=([11]), activation='relu'),
  #  keras.layers.Dense(10, input_shape=([9]), activation='relu'),
    keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=0.01),
              loss='categorical_crossentropy', metrics=['accuracy'])

In [37]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 3)                 36        
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 8         
Total params: 44
Trainable params: 44
Non-trainable params: 0
_________________________________________________________________


In [38]:
hist = model.fit(
    x=training_examples,
    y=to_categorical(training_targets),
    batch_size=100,
    epochs=500,
    validation_split=0.2,
    shuffle=True,
)

Train on 592 samples, validate on 149 samples
Epoch 1/500
592/592 [==============================] - 0s 250us/sample - loss: 0.7579 - acc: 0.4206 - val_loss: 0.6510 - val_acc: 0.6779
Epoch 2/500
592/592 [==============================] - 0s 21us/sample - loss: 0.6315 - acc: 0.6639 - val_loss: 0.5900 - val_acc: 0.7718
Epoch 3/500
592/592 [==============================] - 0s 51us/sample - loss: 0.5869 - acc: 0.7416 - val_loss: 0.5622 - val_acc: 0.7517
Epoch 4/500
592/592 [==============================] - 0s 46us/sample - loss: 0.5583 - acc: 0.7584 - val_loss: 0.5310 - val_acc: 0.7785
Epoch 5/500
592/592 [==============================] - 0s 57us/sample - loss: 0.5298 - acc: 0.7703 - val_loss: 0.4979 - val_acc: 0.7785
Epoch 6/500
592/592 [==============================] - 0s 52us/sample - loss: 0.5064 - acc: 0.7720 - val_loss: 0.4727 - val_acc: 0.7718
Epoch 7/500
592/592 [==============================] - 0s 45us/sample - loss: 0.4906 - acc: 0.7635 - val_loss: 0.4569 - val_acc: 0.7651
E

592/592 [==============================] - 0s 60us/sample - loss: 0.4388 - acc: 0.8125 - val_loss: 0.4726 - val_acc: 0.7584
Epoch 61/500
592/592 [==============================] - 0s 36us/sample - loss: 0.4384 - acc: 0.8125 - val_loss: 0.4723 - val_acc: 0.7584
Epoch 62/500
592/592 [==============================] - 0s 62us/sample - loss: 0.4381 - acc: 0.8125 - val_loss: 0.4749 - val_acc: 0.7584
Epoch 63/500
592/592 [==============================] - 0s 44us/sample - loss: 0.4384 - acc: 0.8125 - val_loss: 0.4756 - val_acc: 0.7584
Epoch 64/500
592/592 [==============================] - 0s 35us/sample - loss: 0.4386 - acc: 0.8125 - val_loss: 0.4771 - val_acc: 0.7584
Epoch 65/500
592/592 [==============================] - 0s 41us/sample - loss: 0.4382 - acc: 0.8108 - val_loss: 0.4798 - val_acc: 0.7584
Epoch 66/500
592/592 [==============================] - 0s 42us/sample - loss: 0.4381 - acc: 0.8091 - val_loss: 0.4787 - val_acc: 0.7584
Epoch 67/500
592/592 [==============================] 

Epoch 120/500
592/592 [==============================] - 0s 34us/sample - loss: 0.4354 - acc: 0.8142 - val_loss: 0.4887 - val_acc: 0.7651
Epoch 121/500
592/592 [==============================] - 0s 67us/sample - loss: 0.4352 - acc: 0.8142 - val_loss: 0.4897 - val_acc: 0.7651
Epoch 122/500
592/592 [==============================] - 0s 38us/sample - loss: 0.4357 - acc: 0.8125 - val_loss: 0.4909 - val_acc: 0.7651
Epoch 123/500
592/592 [==============================] - 0s 69us/sample - loss: 0.4350 - acc: 0.8142 - val_loss: 0.4885 - val_acc: 0.7651
Epoch 124/500
592/592 [==============================] - 0s 52us/sample - loss: 0.4357 - acc: 0.8142 - val_loss: 0.4855 - val_acc: 0.7651
Epoch 125/500
592/592 [==============================] - 0s 64us/sample - loss: 0.4352 - acc: 0.8142 - val_loss: 0.4866 - val_acc: 0.7651
Epoch 126/500
592/592 [==============================] - 0s 38us/sample - loss: 0.4351 - acc: 0.8142 - val_loss: 0.4867 - val_acc: 0.7651
Epoch 127/500
592/592 [===========

592/592 [==============================] - 0s 44us/sample - loss: 0.4347 - acc: 0.8159 - val_loss: 0.4936 - val_acc: 0.7651
Epoch 180/500
592/592 [==============================] - 0s 33us/sample - loss: 0.4344 - acc: 0.8142 - val_loss: 0.4890 - val_acc: 0.7584
Epoch 181/500
592/592 [==============================] - 0s 36us/sample - loss: 0.4342 - acc: 0.8142 - val_loss: 0.4894 - val_acc: 0.7584
Epoch 182/500
592/592 [==============================] - 0s 56us/sample - loss: 0.4344 - acc: 0.8142 - val_loss: 0.4930 - val_acc: 0.7584
Epoch 183/500
592/592 [==============================] - 0s 50us/sample - loss: 0.4349 - acc: 0.8176 - val_loss: 0.4909 - val_acc: 0.7584
Epoch 184/500
592/592 [==============================] - 0s 40us/sample - loss: 0.4344 - acc: 0.8159 - val_loss: 0.4899 - val_acc: 0.7584
Epoch 185/500
592/592 [==============================] - 0s 57us/sample - loss: 0.4352 - acc: 0.8159 - val_loss: 0.4904 - val_acc: 0.7584
Epoch 186/500
592/592 [=========================

592/592 [==============================] - 0s 50us/sample - loss: 0.4348 - acc: 0.8142 - val_loss: 0.4871 - val_acc: 0.7584
Epoch 239/500
592/592 [==============================] - 0s 46us/sample - loss: 0.4333 - acc: 0.8108 - val_loss: 0.4891 - val_acc: 0.7517
Epoch 240/500
592/592 [==============================] - 0s 57us/sample - loss: 0.4339 - acc: 0.8057 - val_loss: 0.4852 - val_acc: 0.7584
Epoch 241/500
592/592 [==============================] - 0s 47us/sample - loss: 0.4342 - acc: 0.8091 - val_loss: 0.4803 - val_acc: 0.7584
Epoch 242/500
592/592 [==============================] - 0s 72us/sample - loss: 0.4339 - acc: 0.8125 - val_loss: 0.4831 - val_acc: 0.7584
Epoch 243/500
592/592 [==============================] - 0s 34us/sample - loss: 0.4332 - acc: 0.8091 - val_loss: 0.4863 - val_acc: 0.7584
Epoch 244/500
592/592 [==============================] - 0s 45us/sample - loss: 0.4329 - acc: 0.8091 - val_loss: 0.4849 - val_acc: 0.7584
Epoch 245/500
592/592 [=========================

592/592 [==============================] - 0s 48us/sample - loss: 0.4338 - acc: 0.8108 - val_loss: 0.4807 - val_acc: 0.7517
Epoch 298/500
592/592 [==============================] - 0s 33us/sample - loss: 0.4318 - acc: 0.8108 - val_loss: 0.4826 - val_acc: 0.7584
Epoch 299/500
592/592 [==============================] - 0s 46us/sample - loss: 0.4318 - acc: 0.8091 - val_loss: 0.4851 - val_acc: 0.7584
Epoch 300/500
592/592 [==============================] - 0s 34us/sample - loss: 0.4317 - acc: 0.8091 - val_loss: 0.4840 - val_acc: 0.7517
Epoch 301/500
592/592 [==============================] - 0s 32us/sample - loss: 0.4327 - acc: 0.8108 - val_loss: 0.4831 - val_acc: 0.7517
Epoch 302/500
592/592 [==============================] - 0s 36us/sample - loss: 0.4315 - acc: 0.8125 - val_loss: 0.4813 - val_acc: 0.7517
Epoch 303/500
592/592 [==============================] - 0s 33us/sample - loss: 0.4320 - acc: 0.8108 - val_loss: 0.4796 - val_acc: 0.7517
Epoch 304/500
592/592 [=========================

592/592 [==============================] - 0s 26us/sample - loss: 0.4307 - acc: 0.8108 - val_loss: 0.4828 - val_acc: 0.7517
Epoch 357/500
592/592 [==============================] - 0s 69us/sample - loss: 0.4316 - acc: 0.8108 - val_loss: 0.4806 - val_acc: 0.7517
Epoch 358/500
592/592 [==============================] - 0s 47us/sample - loss: 0.4311 - acc: 0.8108 - val_loss: 0.4840 - val_acc: 0.7517
Epoch 359/500
592/592 [==============================] - 0s 49us/sample - loss: 0.4310 - acc: 0.8108 - val_loss: 0.4785 - val_acc: 0.7517
Epoch 360/500
592/592 [==============================] - 0s 45us/sample - loss: 0.4318 - acc: 0.8108 - val_loss: 0.4745 - val_acc: 0.7517
Epoch 361/500
592/592 [==============================] - 0s 30us/sample - loss: 0.4304 - acc: 0.8108 - val_loss: 0.4817 - val_acc: 0.7517
Epoch 362/500
592/592 [==============================] - 0s 49us/sample - loss: 0.4324 - acc: 0.8057 - val_loss: 0.4825 - val_acc: 0.7517
Epoch 363/500
592/592 [=========================

592/592 [==============================] - 0s 40us/sample - loss: 0.4311 - acc: 0.8108 - val_loss: 0.4773 - val_acc: 0.7517
Epoch 416/500
592/592 [==============================] - 0s 40us/sample - loss: 0.4313 - acc: 0.8142 - val_loss: 0.4785 - val_acc: 0.7517
Epoch 417/500
592/592 [==============================] - 0s 56us/sample - loss: 0.4299 - acc: 0.8142 - val_loss: 0.4794 - val_acc: 0.7517
Epoch 418/500
592/592 [==============================] - 0s 47us/sample - loss: 0.4304 - acc: 0.8142 - val_loss: 0.4789 - val_acc: 0.7517
Epoch 419/500
592/592 [==============================] - 0s 42us/sample - loss: 0.4302 - acc: 0.8142 - val_loss: 0.4821 - val_acc: 0.7517
Epoch 420/500
592/592 [==============================] - 0s 35us/sample - loss: 0.4303 - acc: 0.8142 - val_loss: 0.4843 - val_acc: 0.7517
Epoch 421/500
592/592 [==============================] - 0s 30us/sample - loss: 0.4295 - acc: 0.8142 - val_loss: 0.4818 - val_acc: 0.7517
Epoch 422/500
592/592 [=========================

592/592 [==============================] - 0s 42us/sample - loss: 0.4288 - acc: 0.8159 - val_loss: 0.4734 - val_acc: 0.7651
Epoch 475/500
592/592 [==============================] - 0s 43us/sample - loss: 0.4288 - acc: 0.8176 - val_loss: 0.4808 - val_acc: 0.7651
Epoch 476/500
592/592 [==============================] - 0s 44us/sample - loss: 0.4284 - acc: 0.8125 - val_loss: 0.4794 - val_acc: 0.7651
Epoch 477/500
592/592 [==============================] - 0s 47us/sample - loss: 0.4291 - acc: 0.8159 - val_loss: 0.4786 - val_acc: 0.7651
Epoch 478/500
592/592 [==============================] - 0s 59us/sample - loss: 0.4291 - acc: 0.8142 - val_loss: 0.4814 - val_acc: 0.7651
Epoch 479/500
592/592 [==============================] - 0s 61us/sample - loss: 0.4280 - acc: 0.8125 - val_loss: 0.4805 - val_acc: 0.7651
Epoch 480/500
592/592 [==============================] - 0s 39us/sample - loss: 0.4281 - acc: 0.8142 - val_loss: 0.4832 - val_acc: 0.7651
Epoch 481/500
592/592 [=========================

In [39]:
model.evaluate(testing_examples, to_categorical(testing_targets))

150/150 [==============================] - 0s 53us/sample - loss: 0.4025 - acc: 0.8267


[0.40251835068066916, 0.82666665]

In [ ]:
# Best testing score so far:
# [0.40251835068066916, 0.82666665]